In [3]:
""" A Mobile Net model is imported from TFHub and it is used for predictions for cats_vs_dogs dataset. Finally, 
it will be saved as TFLite model after Optimizations and Interpretations. """

# Imports
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [2]:
# Getting model from TFHub. Inception model can also be used instead of MobileNet.

module_selection = ("mobilenet_v2", 224, 1280) #@param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(
  MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

Using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 with input size (224, 224) and output dimension 1280


In [17]:
# Building model and data
splits = tfds.Split.ALL.subsplit(weighted=(80,10,10))
splits, info = tfds.load("cats_vs_dogs", with_info = True, as_supervised = True, split=splits)
(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

def format_image(image, label):
    image = tf.image.resize(image,IMAGE_SIZE) / 255.0
    return image, label

BATCH_SIZE = 32

train_batches = train_examples.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

feature_extractor = hub.KerasLayer(MODULE_HANDLE, input_shape = IMAGE_SIZE + (3,), output_shape = [FV_SIZE],
                                  trainable = False)

model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(units=num_classes, activation='softmax')
])
model.summary()

model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 1280)              2257984   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
# Training(Not done here)
EPOCHS=5
history = model.fit(train_batches, validation_data=validation_batches, epochs=EPOCHS)

# Exporting model
CATS_VS_DOGS_SAVED_MODEL = 'exp_saved_model'
tf.saved_model().save(model,CATS_VS_DOGS_SAVED_MODEL)

loaded = tf.saved_model.load(CATS_VS_DOGS_SAVED_MODEL)
print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signatures)
print(infer.structured_outputs)

In [ ]:
# Converter
converter = tf.lite.TFLiteConverter(CATS_VS_DOGS_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Representative Data Generator
def representative_data_gen():
    for input_value, _ in test_batches.take(100):
        yield [input_value]
        
converter.representative_dataset = representative_data_gen

converter.target_specs.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Converted Model
tflite_model = converter.convert()
tflite_model_file = 'converted_model.tflite'

with open(tflite_model_file,'wb') as f:
    f.write(tflite_model)

In [ ]:
# Interpreting data
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]['index']
output_details = interpreter.get_output_details()[0]['index']

from tqdm import tqdm
predictions = []
test_labels, test_imgs = [], []
for img,label in tqdm(test_batches.take(10)):
    interpreter.set_tensor(input_details, img)
    interpreter.invoke()
    predictions.append(interpreter.get_tensor(output_details))
    
    test_labels.append(label.numpy()[0])
    test_imgs.append(img)

In [ ]:
# Writing outputs and downloading it with tfLite model
labels = ['cat','dog']
with open('labels.txt','w') as f:
    f.write('\n'.join(labels))

from google.colab import files
files.download('converted_model.tflite')
files.download('labels.txt')